In [3]:
import pandas as pd
df = pd.read_csv(r'C:/Users/jgaur/python_ml/Handle-Imbalance-Data/creditcard.csv')

In [4]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
df.Class.value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [6]:
## Independent and dependent features
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

### Cross Validation: KFold and Hyperparameter Technique: GridSearchCV

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import KFold
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [8]:
log_class = LogisticRegression()
grid = {'C': 10.0 ** np.arange(-2, 3), 'penalty': ['l1', 'l2']}
cv = KFold(n_splits=5, shuffle=False, random_state=None)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7)

In [10]:
clf = GridSearchCV(log_class, grid, cv=cv, n_jobs=-1, scoring='f1_macro')
clf.fit(X_train, y_train)

C:\Users\jgaur\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan 0.80916589        nan 0.82523728        nan 0.82316692
        nan 0.82439878        nan 0.82864731]
  warnings.warn(
C:\Users\jgaur\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'penalty': ['l1', 'l2']},
             scoring='f1_macro')

In [11]:
prediction = clf.predict(X_test)
print(confusion_matrix(y_test, prediction))
print(accuracy_score(y_test, prediction))
print(classification_report(y_test, prediction))

[[198927    100]
 [    96    242]]
0.9990168785895217
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    199027
           1       0.71      0.72      0.71       338

    accuracy                           1.00    199365
   macro avg       0.85      0.86      0.86    199365
weighted avg       1.00      1.00      1.00    199365



In [12]:
class_weight = dict({0: 1, 1: 100})

In [13]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(class_weight=class_weight)
clf.fit(X_train, y_train)

RandomForestClassifier(class_weight={0: 1, 1: 100})

In [14]:
prediction = clf.predict(X_test)
print(confusion_matrix(y_test, prediction))
print(accuracy_score(y_test, prediction))
print(classification_report(y_test, prediction))

[[199005     22]
 [    86    252]]
0.9994582800391242
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    199027
           1       0.92      0.75      0.82       338

    accuracy                           1.00    199365
   macro avg       0.96      0.87      0.91    199365
weighted avg       1.00      1.00      1.00    199365



### Under Sampling

In [15]:
from imblearn.under_sampling import NearMiss
from collections import Counter

In [16]:
ns = NearMiss(0.8)
X_train_ns, y_train_ns = ns.fit_resample(X_train, y_train)
print("The number of classes before fit {}".format(Counter(y_train)))
print("The number of classes after fit {}".format(Counter(y_train_ns)))

C:\Users\jgaur\anaconda3\lib\site-packages\imblearn\utils\_validation.py:587: FutureWarning: Pass sampling_strategy=0.8 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn(


The number of classes before fit Counter({0: 85288, 1: 154})
The number of classes after fit Counter({0: 192, 1: 154})


In [17]:
print("The number of classes before fit {}".format(Counter(y_train)))
print("The number of classes after fit {}".format(Counter(y_train_ns)))

The number of classes before fit Counter({0: 85288, 1: 154})
The number of classes after fit Counter({0: 192, 1: 154})


In [18]:
y_train.value_counts()

0    85288
1      154
Name: Class, dtype: int64

In [19]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train_ns, y_train_ns)

RandomForestClassifier()

In [20]:
prediction = clf.predict(X_test)
print(confusion_matrix(y_test, prediction))
print(accuracy_score(y_test, prediction))
print(classification_report(y_test, prediction))

[[120874  78153]
 [    15    323]]
0.6079151305394628
              precision    recall  f1-score   support

           0       1.00      0.61      0.76    199027
           1       0.00      0.96      0.01       338

    accuracy                           0.61    199365
   macro avg       0.50      0.78      0.38    199365
weighted avg       1.00      0.61      0.75    199365



### Over Sampling

In [21]:
from imblearn.over_sampling import RandomOverSampler

In [25]:
os = RandomOverSampler(0.5)
X_train_os, y_train_os = os.fit_resample(X_train, y_train)
print("The number of classes before fit {}".format(Counter(y_train)))
print("The number of classes after fit {}".format(Counter(y_train_os)))

C:\Users\jgaur\anaconda3\lib\site-packages\imblearn\utils\_validation.py:587: FutureWarning: Pass sampling_strategy=0.5 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn(


The number of classes before fit Counter({0: 85288, 1: 154})
The number of classes after fit Counter({0: 85288, 1: 42644})


In [26]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train_os, y_train_os)

RandomForestClassifier()

In [27]:
prediction = clf.predict(X_test)
print(confusion_matrix(y_test, prediction))
print(accuracy_score(y_test, prediction))
print(classification_report(y_test, prediction))

[[199005     22]
 [    82    256]]
0.9994783437413789
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    199027
           1       0.92      0.76      0.83       338

    accuracy                           1.00    199365
   macro avg       0.96      0.88      0.92    199365
weighted avg       1.00      1.00      1.00    199365



### SMOTETomek

In [28]:
from imblearn.combine import SMOTETomek

In [29]:
os = SMOTETomek(0.5)
X_train_os, y_train_os = os.fit_resample(X_train, y_train)
print("The number of classes before fit {}".format(Counter(y_train)))
print("The number of classes after fit {}".format(Counter(y_train_os)))

C:\Users\jgaur\anaconda3\lib\site-packages\imblearn\utils\_validation.py:587: FutureWarning: Pass sampling_strategy=0.5 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn(


The number of classes before fit Counter({0: 85288, 1: 154})
The number of classes after fit Counter({0: 84508, 1: 41864})


In [30]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train_os, y_train_os)

RandomForestClassifier()

In [31]:
prediction = clf.predict(X_test)
print(confusion_matrix(y_test, prediction))
print(accuracy_score(y_test, prediction))
print(classification_report(y_test, prediction))

[[198981     46]
 [    61    277]]
0.9994632959646879
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    199027
           1       0.86      0.82      0.84       338

    accuracy                           1.00    199365
   macro avg       0.93      0.91      0.92    199365
weighted avg       1.00      1.00      1.00    199365

